In [4]:
# Hellooo
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

file_path = '/content/drive/My Drive/Network Science Project/cleaned_airports.csv'
df = pd.read_csv(file_path)
routes_file_path = '/content/drive/My Drive/Network Science Project/cleaned_routes.csv'
routes_df = pd.read_csv(routes_file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
import networkx as nx
# Create a new graph object
graph = nx.Graph()

# Add nodes based on airport data
for index, row in df.iterrows():
    graph.add_node(row['IATA'],
                   name=row['Name'],
                   city=row['City'],
                   country=row['Country'],
                   latitude=row['Latitude'],
                   longitude=row['Longitude'])

# Add edges based on routes data
for index, row in routes_df.iterrows():
    source_airport = row['Source']
    destination_airport = row['Destination']
    # Check if both airports exist in the graph's nodes
    if source_airport in graph.nodes and destination_airport in graph.nodes:
        graph.add_edge(source_airport, destination_airport, airline=row['Airline'])

# Robustness

In [9]:
import random

def test_robustness(graph, removal_strategy, fraction_to_remove):
    """
    Tests network robustness by removing nodes/edges.

    Args:
        graph: The networkx graph object.
        removal_strategy: 'high_degree', 'low_degree', 'random'.
        fraction_to_remove: Fraction of nodes/edges to remove.

    Returns:
        The size of the largest connected component after removal.
    """
    graph_copy = graph.copy()  # Create a copy to avoid modifying the original graph
    num_to_remove = int(len(graph_copy) * fraction_to_remove)

    if removal_strategy == 'high_degree':
        nodes_to_remove = sorted(graph_copy.degree, key=lambda item: item[1], reverse=True)[:num_to_remove]
        nodes_to_remove = [node[0] for node in nodes_to_remove]
        graph_copy.remove_nodes_from(nodes_to_remove)

    elif removal_strategy == 'low_degree':
        nodes_to_remove = sorted(graph_copy.degree, key=lambda item: item[1])[:num_to_remove]
        nodes_to_remove = [node[0] for node in nodes_to_remove]
        graph_copy.remove_nodes_from(nodes_to_remove)

    elif removal_strategy == 'random':
        nodes_to_remove = random.sample(list(graph_copy.nodes()), num_to_remove)
        graph_copy.remove_nodes_from(nodes_to_remove)

    # Calculate largest connected component size
    largest_cc = max(nx.connected_components(graph_copy), key=len)
    return len(largest_cc)

# usage
fraction_removed = 0.1  # Remove 10% of nodes
robustness_high_degree = test_robustness(graph, 'high_degree', fraction_removed)
robustness_low_degree = test_robustness(graph, 'low_degree', fraction_removed)
robustness_random = test_robustness(graph, 'random', fraction_removed)

# Calculate baseline LCC before removal
baseline_lcc_size = len(max(nx.connected_components(graph), key=len))


print(f"Baseline LCC size: {baseline_lcc_size}")
print(f"Robustness (High Degree Removal): {robustness_high_degree}")
print(f"Robustness (Low Degree Removal): {robustness_low_degree}")
print(f"Robustness (Random Removal): {robustness_random}")

Baseline LCC size: 3231
Robustness (High Degree Removal): 92
Robustness (Low Degree Removal): 3231
Robustness (Random Removal): 2791


# Delay Propagation

In [17]:
import random

def simulate_delay_propagation(graph, initial_delay_airports, delay_duration):
    """Simulates delay propagation in the network."""
    # Initialize delay dictionary
    delays = {airport: 0 for airport in graph.nodes()}

    # Introduce initial delays
    for airport in initial_delay_airports:
        delays[airport] = delay_duration

    # Simulate delay propagation for a fixed number of steps
    for _ in range(10):  # Adjust the number of steps as needed
        for airport in graph.nodes():
            if delays[airport] > 0:
                # Spread delay to neighbors
                for neighbor in graph.neighbors(airport):
                    # Assume delay propagation with some probability
                    if random.random() < 0.5:  # Adjust probability as needed
                        delays[neighbor] = max(delays[neighbor], delays[airport] - 1)  # Delay decreases over time
                delays[airport] -= 1  # Delay at current airport decreases

    return delays

# usage
initial_delay_airports = ['JFK', 'ORD']  # Replace with actual airport codes
delay_duration = 60  # Initial delay in minutes
final_delays = simulate_delay_propagation(graph, initial_delay_airports, delay_duration)

# Print degree of airports with initial delay
for airport in initial_delay_airports:
    degree = graph.degree(airport)
    print(f" Delayed Airport: {airport}, Degree: {degree}")

# Calculate number and percentage of airports with delays
delayed_airports = [airport for airport, delay in final_delays.items() if delay > 0]
num_delayed_airports = len(delayed_airports)
total_airports = len(graph.nodes())
percentage_delayed = (num_delayed_airports / total_airports) * 100

print(f"\nNumber of airports experiencing delays: {num_delayed_airports}")
print(f"Percentage of airports experiencing delays: {percentage_delayed:.2f}%")

# Calculate average, min, and max delay
delays_list = [delay for delay in final_delays.values() if delay > 0]  # Exclude 0 delays
average_delay = sum(delays_list) / len(delays_list) if delays_list else 0  # Handle empty list
min_delay = min(delays_list) if delays_list else 0
max_delay = max(delays_list) if delays_list else 0

print(f"\nAverage delay: {average_delay:.2f} minutes")
# Print delays with full airport names
print("\nAirport Delays:")
# Find airport with minimum delay
min_delay_airport = min(final_delays, key=final_delays.get) if delays_list else None
if min_delay_airport:
    min_delay_airport_name = next((node_data['name'] for node, node_data in graph.nodes(data=True) if node == min_delay_airport), min_delay_airport)
    print(f"Airport with minimum delay: {min_delay_airport_name} ({min_delay} minutes)")

# Find airport with maximum delay
max_delay_airport = max(final_delays, key=final_delays.get) if delays_list else None
if max_delay_airport:
    max_delay_airport_name = next((node_data['name'] for node, node_data in graph.nodes(data=True) if node == max_delay_airport), max_delay_airport)
    print(f"Airport with maximum delay: {max_delay_airport_name} ({max_delay} minutes)")




 Delayed Airport: JFK, Degree: 162
 Delayed Airport: ORD, Degree: 206

Number of airports experiencing delays: 3223
Percentage of airports experiencing delays: 53.07%

Average delay: 49.33 minutes

Airport Delays:
Airport with minimum delay: Hornafjörður Airport (46 minutes)
Airport with maximum delay: Goroka Airport (50 minutes)


In [19]:
def find_critical_airports(graph, delay_duration):
    """Finds the top airports causing the most delays."""
    airport_delays = {}  # Store average delays for each airport

    for airport in graph.nodes():
        # Simulate delay propagation with a delay at the current airport
        final_delays = simulate_delay_propagation(graph, [airport], delay_duration)

        # Calculate average delay across all airports
        total_delay = sum(final_delays.values())
        avg_delay = total_delay / len(graph.nodes())

        # Store average delay for this airport
        airport_delays[airport] = avg_delay

    # Rank airports by average delay (descending)
    ranked_airports = sorted(airport_delays.items(), key=lambda item: item[1], reverse=True)

    return ranked_airports

delay_duration = 60  # Initial delay in minutes
ranked_airports = find_critical_airports(graph, delay_duration)

print("Top 5 airports causing the most delays:")
for i in range(min(5, len(ranked_airports))):  # Print up to 5 airports
    airport, avg_delay = ranked_airports[i]
    airport_name = next((node_data['name'] for node, node_data in graph.nodes(data=True) if node == airport), airport)
    print(f"{i + 1}. {airport_name}: {avg_delay:.2f} minutes")

Top 5 airports causing the most delays:
1. Istanbul Airport: 26.25 minutes
2. McCarran International Airport: 26.25 minutes
3. Northwest Arkansas Regional Airport: 26.25 minutes
4. Rajiv Gandhi International Airport: 26.24 minutes
5. Provo Municipal Airport: 26.24 minutes


# Ranking

In [25]:
def rank_nodes_by_centrality(graph, weight='weight'):
    """Ranks nodes by combined centrality metrics."""
    # Calculate centrality metrics
    degree_centrality = nx.degree_centrality(graph)
    betweenness_centrality = nx.betweenness_centrality(graph, weight=weight)
    closeness_centrality = nx.closeness_centrality(graph)
    eigenvector_centrality = nx.eigenvector_centrality(graph, weight=weight)

    # Combine centrality metrics (We can adjust weights but i think this is the best)
    combined_centrality = {}
    for node in graph.nodes():
        combined_centrality[node] = (
            0.25 * degree_centrality[node] +
            0.25 * betweenness_centrality[node] +
            0.25 * closeness_centrality[node] +
            0.25 * eigenvector_centrality[node]
        )

    # Rank nodes by combined centrality (descending)
    ranked_nodes = sorted(combined_centrality.items(), key=lambda item: item[1], reverse=True)

    return ranked_nodes

ranked_nodes = rank_nodes_by_centrality(graph)

print("Top 10 critical nodes:")
for i in range(10):
    node, centrality = ranked_nodes[i]
    node_data = graph.nodes[node]

    print(f"{i + 1}. {node_data['name']}")

Top 10 critical nodes:
1. Frankfurt am Main Airport
2. Amsterdam Airport Schiphol
3. Charles de Gaulle International Airport
4. London Heathrow Airport
5. Munich Airport
6. Istanbul Airport
7. Leonardo da Vinci–Fiumicino Airport
8. Dubai International Airport
9. Adolfo Suárez Madrid–Barajas Airport
10. Zürich Airport
